In [2]:
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
from untrade.client import Client
import plotly.graph_objects as go


from scipy.signal import argrelextrema


data = pd.read_csv(
    '/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/BTC/BTC_2019_2023_30m.csv',
    # parse_dates=['datetime'],  # Parse the 'datetime' column as datetime
    # index_col='datetime'       # Use 'datetime' as the index
)

print(data.head())

   Unnamed: 0             datetime      open      high       low     close  \
0           0  2019-09-08 17:30:00  10000.00  10000.00  10000.00  10000.00   
1           1  2019-09-08 18:00:00  10000.00  10000.00  10000.00  10000.00   
2           2  2019-09-08 18:30:00  10000.00  10000.00  10000.00  10000.00   
3           3  2019-09-08 19:00:00  10344.77  10357.53  10342.90  10354.62   
4           4  2019-09-08 19:30:00  10354.62  10357.35  10337.43  10340.12   

    volume  
0    0.002  
1    0.000  
2    0.000  
3  136.177  
4  335.482  


In [3]:
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
import plotly.graph_objects as go

def extract_extremes(price_values, i, order_dec):
    max_idx = list(argrelextrema(price_values[:i], np.greater, order=order_dec)[0])
    min_idx = list(argrelextrema(price_values[:i], np.less, order=order_dec)[0])
    idx = max_idx + min_idx + [len(price_values[:i]) - 1]
    idx.sort()
    current_idx = idx[-5:]
    current_pat = price_values[current_idx]
    return current_idx, current_pat

def identify_pattern(current_idx, current_pat, err_allowed, ab, bc, cd):
    XA, AB, BC, CD = np.diff(current_pat)
    
    AB_range = np.array([ab[0] - err_allowed, ab[1] + err_allowed]) * abs(XA)
    BC_range = np.array([bc[0] - err_allowed, bc[1] + err_allowed]) * abs(AB)
    CD_range = np.array([cd[0] - err_allowed, cd[1] + err_allowed]) * abs(BC)

    abs_AB, abs_BC, abs_CD = map(abs, (AB, BC, CD))

    if XA > 0 and AB < 0 and BC > 0 and CD < 0:
        if AB_range[0] < abs_AB < AB_range[1] and BC_range[0] < abs_BC < BC_range[1] and CD_range[0] < abs_CD < CD_range[1]:
            return 1, current_idx
    elif XA < 0 and AB > 0 and BC < 0 and CD > 0:
        if AB_range[0] < abs_AB < AB_range[1] and BC_range[0] < abs_BC < BC_range[1] and CD_range[0] < abs_CD < CD_range[1]:
            return -1, current_idx
    return np.nan, []

harmonic_patterns = {
    "Butterfly": ([0.786, 0.786], [0.382, 0.886], [1.618, 2.618]),
    "Gartley": ([0.618, 0.618], [0.382, 0.886], [1.272, 1.618]),
    "Cypher": ([0.382, 0.618], [1.272, 1.414], [0.786, 1.786]),
    "Bat": ([0.382, 0.5], [0.382, 0.886], [1.618, 2.618]),
    "AltBat": ([0.382, 0.382], [0.382, 0.886], [2.0, 3.618]),
    "Crab": ([0.382, 0.618], [0.382, 0.886], [2.24, 3.618]),
    "DeepCrab": ([0.886, 0.886], [0.382, 0.886], [2.618, 3.618]),
    "Shark": ([0.382, 0.886], [1.13, 1.618], [1.618, 2.24]),
    "WhiteSwan": ([1.382, 2.618], [0.236, 0.5], [1.128, 2.0]),
    "BlackSwan": ([0.382, 0.724], [2.0, 4.237], [0.5, 0.886])
}

def detect_harmonic_pattern(price, order_dec=5, err_allowed=0.1):
    signals = {pattern: [] for pattern in harmonic_patterns}
    patterns = {pattern: [] for pattern in harmonic_patterns}
    
    for i in range(100, len(price)):
        current_idx, current_pat = extract_extremes(price.values, i, order_dec)

        for pattern_name, ratios in harmonic_patterns.items():
            pattern_signal, pattern_idx = identify_pattern(current_idx, current_pat, err_allowed, *ratios)
            signals[pattern_name].append(pattern_signal)
            if not np.isnan(pattern_signal):
                patterns[pattern_name].append((pattern_signal, pattern_idx))
    
    return signals, patterns

def plot_harmonic_patterns(df, patterns):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(x=df.index, 
                                 open=df['open'], 
                                 high=df['high'], 
                                 low=df['low'], 
                                 close=df['close'], 
                                 name='Candlesticks'))
    
    colors = {
        "Butterfly": "blue", "Bat": "green", "AltBat": "purple",
        "Crab": "red", "DeepCrab": "pink", "Cypher": "orange",
        "Gartley": "cyan", "Shark": "yellow", "WhiteSwan": "black", "BlackSwan": "brown"
    }
    
    for pattern_name, pattern_list in patterns.items():
        for signal, indices in pattern_list:
            x_vals = df.index[indices]
            y_vals = df.close.iloc[indices]
            fig.add_trace(go.Scatter(
                x=x_vals, y=y_vals, mode='lines+markers',
                line=dict(color=colors[pattern_name], width=2),
                marker=dict(size=8, color=colors[pattern_name]),
                name=f"{pattern_name} {'Bullish' if signal == 1 else 'Bearish'}"
            ))

    fig.update_layout(title='Harmonic Patterns',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False)
    fig.show()


# Detect harmonic patterns
signals, patterns = detect_harmonic_pattern(data['close'])

# Plot the patterns
# plot_harmonic_patterns(data, patterns)


In [4]:
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
def WMA(data, window_size):
    weights = np.arange(1, window_size + 1)
    return data.rolling(window=window_size).apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)

def HMA(data, window_size):
    return WMA(2 * WMA(data, window_size//2) - WMA(data, window_size), int(np.sqrt(window_size)))

def plot_harmonic_patterns(df, patterns):
    output_dict = {}
    period  = 26
    hma = HMA(df['close'], period)

    for pattern_name, pattern_list in patterns.items():
        if not pattern_list:
            continue  # Skip patterns that have no occurrences

        slider = widgets.IntSlider(min=0, max=len(pattern_list) - 1, step=1, description=f"{pattern_name} Index")
        output = widgets.Output()
        output_dict[pattern_name] = (slider, output)

        def update_plot(pattern_idx, pattern_name=pattern_name):
            with output_dict[pattern_name][1]:
                clear_output()
                signal, indices = patterns[pattern_name][pattern_idx]

                start = max(indices[0] - 20, 0)
                end = min(indices[-1] + 20, len(df) - 1)

                fig = go.Figure()
                fig.add_trace(go.Candlestick(
                    x=df.index[start:end],
                    open=df['open'][start:end],
                    high=df['high'][start:end],
                    low=df['low'][start:end],
                    close=df['close'][start:end],
                    name='Candlesticks'
                ))

                fig.add_trace(go.Scatter(
                    x=df.index[indices], y=df.close.iloc[indices], mode='lines+markers',
                    line=dict(color='blue' if signal == 1 else 'red', width=2),
                    marker=dict(size=8, color='blue' if signal == 1 else 'red'),
                    name=f"{pattern_name} {'Bullish' if signal == 1 else 'Bearish'}"
                ))

                # Add HMA
                fig.add_trace(go.Scatter(
                    x=df.index[start:end],
                    y=hma[start:end],
                    mode='lines',
                    line=dict(color='purple', width=1),
                    name=f'HMA ({period})'
                ))

                fig.update_layout(
                    title=f'{pattern_name} Pattern - Occurrence {pattern_idx + 1}',
                    xaxis_title='Date',
                    yaxis_title='Price',
                    xaxis_rangeslider_visible=False
                )

                fig.show()

        display(widgets.Label(f"Pattern: {pattern_name}"), slider, output)
        widgets.interactive(update_plot, pattern_idx=slider)
        update_plot(0)

# Example usage
plot_harmonic_patterns(data, patterns)

Label(value='Pattern: Butterfly')

IntSlider(value=0, description='Butterfly Index', max=255)

Output()

Label(value='Pattern: Gartley')

IntSlider(value=0, description='Gartley Index', max=203)

...

IntSlider(value=0, description='WhiteSwan Index', max=408)

Output()

Label(value='Pattern: BlackSwan')

IntSlider(value=0, description='BlackSwan Index', max=968)

Output()